# ДЗ №6

Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

### Загрузка данных

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import warnings

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.layers import SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping


warnings.filterwarnings('ignore')

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
data = pd.read_csv("../lesson05/отзывы за лето.csv", sep=';')
data = data[data['Rating'] != 3]
data['Content'] = data['Content'].apply(preprocess_text)
data['Rating'] = data['Rating'] > 3
data['Rating'] = data['Rating'].astype(int)

In [9]:
train, test = train_test_split(data,  random_state=42, test_size=0.2)
train, val  = train_test_split(train, random_state=42, test_size=0.2)

In [10]:
text_corpus_train = train['Content'].values
text_corpus_valid = val['Content'].values
text_corpus_test = test['Content'].values

In [11]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

y_train = train['Rating'].values
y_val = val['Rating'].values
y_test = test['Rating'].values

In [12]:
def evaluate(model):
    early_stopping=EarlyStopping(monitor='val_loss', patience=3)  

    history = model.fit(X_train, y_train, batch_size=512, epochs=20, verbose=1,
                        validation_data=(X_valid, y_val),
                        callbacks=[early_stopping])

    score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
    print('\n')
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

### Simple RNN

In [13]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

evaluate(model)

Train on 12638 samples, validate on 3160 samples
Epoch 1/20
12638/12638 [==============================] - 9s 740us/step - loss: 0.4731 - accuracy: 0.8310 - val_loss: 0.3834 - val_accuracy: 0.8494
Epoch 2/20
12638/12638 [==============================] - 7s 522us/step - loss: 0.3347 - accuracy: 0.8677 - val_loss: 0.2806 - val_accuracy: 0.8858
Epoch 3/20
12638/12638 [==============================] - 7s 525us/step - loss: 0.2495 - accuracy: 0.9046 - val_loss: 0.2328 - val_accuracy: 0.9044
Epoch 4/20
12638/12638 [==============================] - 6s 513us/step - loss: 0.1838 - accuracy: 0.9316 - val_loss: 0.2110 - val_accuracy: 0.9120
Epoch 5/20
12638/12638 [==============================] - 7s 516us/step - loss: 0.1297 - accuracy: 0.9547 - val_loss: 0.2046 - val_accuracy: 0.9222
Epoch 6/20
12638/12638 [==============================] - 6s 514us/step - loss: 0.0932 - accuracy: 0.9675 - val_loss: 0.2249 - val_accuracy: 0.9149
Epoch 7/20
12638/12638 [==============================] - 7s 54

### LSTM

In [14]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

evaluate(model)

Train on 12638 samples, validate on 3160 samples
Epoch 1/20
12638/12638 [==============================] - 21s 2ms/step - loss: 0.5248 - accuracy: 0.8324 - val_loss: 0.3892 - val_accuracy: 0.8494
Epoch 2/20
12638/12638 [==============================] - 18s 1ms/step - loss: 0.3479 - accuracy: 0.8508 - val_loss: 0.2865 - val_accuracy: 0.8741
Epoch 3/20
12638/12638 [==============================] - 19s 1ms/step - loss: 0.2589 - accuracy: 0.8978 - val_loss: 0.2372 - val_accuracy: 0.8997
Epoch 4/20
12638/12638 [==============================] - 18s 1ms/step - loss: 0.2052 - accuracy: 0.9212 - val_loss: 0.2084 - val_accuracy: 0.9076
Epoch 5/20
12638/12638 [==============================] - 18s 1ms/step - loss: 0.1632 - accuracy: 0.9374 - val_loss: 0.2012 - val_accuracy: 0.9146
Epoch 6/20
12638/12638 [==============================] - 18s 1ms/step - loss: 0.1312 - accuracy: 0.9520 - val_loss: 0.2061 - val_accuracy: 0.9190
Epoch 7/20
12638/12638 [==============================] - 18s 1ms/ste

### GRU

In [15]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

evaluate(model)

Train on 12638 samples, validate on 3160 samples
Epoch 1/20
12638/12638 [==============================] - 17s 1ms/step - loss: 0.5493 - accuracy: 0.8385 - val_loss: 0.4936 - val_accuracy: 0.8494
Epoch 2/20
12638/12638 [==============================] - 16s 1ms/step - loss: 0.3747 - accuracy: 0.8463 - val_loss: 0.3422 - val_accuracy: 0.8494
Epoch 3/20
12638/12638 [==============================] - 15s 1ms/step - loss: 0.2766 - accuracy: 0.8611 - val_loss: 0.2726 - val_accuracy: 0.8946
Epoch 4/20
12638/12638 [==============================] - 15s 1ms/step - loss: 0.2108 - accuracy: 0.9190 - val_loss: 0.2273 - val_accuracy: 0.9076
Epoch 5/20
12638/12638 [==============================] - 15s 1ms/step - loss: 0.1671 - accuracy: 0.9369 - val_loss: 0.2139 - val_accuracy: 0.9063
Epoch 6/20
12638/12638 [==============================] - 16s 1ms/step - loss: 0.1398 - accuracy: 0.9494 - val_loss: 0.2154 - val_accuracy: 0.9057
Epoch 7/20
12638/12638 [==============================] - 16s 1ms/ste

## Вывод

* можно сделать небольшой вывод о том что сети LSTM или GRU точно работают лучше чем Simple RNN
* также возможно мне пказалось но с LSTM будет больше возможностей добиться большего качества 